In [1]:
# Импорт необходимых библиотек
import numpy as np
import pandas as pd
import tensorflow as tf
from keras_flops import get_flops

In [2]:
# Глобальные константы
CLASSES       = 100
BATCH_SIZE    = 128
LEARNING_RATE = 1e-2

In [3]:
# Выполните загрузку модели
(X_train, y_train), (X_val, y_val) = tf.keras.datasets.cifar100.load_data()

In [4]:
# Преобразуйте метки классов в one_hot формат
y_train = np.eye(CLASSES)[y_train.reshape(-1)]
y_val = np.eye(CLASSES)[y_val.reshape(-1)]

In [5]:
# убедитесь, что данная ячейка выполняется без ошибок
assert X_train.shape == (50000, 32, 32, 3)
assert X_val.shape == (10000, 32, 32, 3)
assert y_train.shape == (50000, 100)
assert y_val.shape == (10000, 100)

In [16]:
# Задайте архитектуру модели

input_shape = (32, 32, 3)

model = tf.keras.models.Sequential()

model.add(tf.keras.layers.SeparableConv2D(6, 3, padding='same', input_shape=input_shape))
model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.LeakyReLU(alpha=0.01))
model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2), strides=(2, 2), padding="same"))

model.add(tf.keras.layers.SeparableConv2D(12, 3, padding='same'))
model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.LeakyReLU(alpha=0.01))
model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2), strides=(2, 2), padding="same"))

model.add(tf.keras.layers.SeparableConv2D(24, 3, padding='same'))
model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.LeakyReLU(alpha=0.01))
model.add(tf.keras.layers.MaxPooling2D(pool_size=(4, 4), strides=(4, 4), padding="same"))

model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(1024))
model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.LeakyReLU(alpha=0.01))
model.add(tf.keras.layers.Dropout(0.5))
model.add(tf.keras.layers.Dense(CLASSES))
model.add(tf.keras.layers.Activation('softmax'))

In [17]:
# вычисление количества операций
flops = get_flops(model, batch_size=1)
print(f"FLOPs: {(flops / 1e6):.4f}e6")

FLOPs: 0.6618e6


In [18]:
# вывод краткой информации о модели
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 separable_conv2d_6 (Separab  (None, 32, 32, 6)        51        
 leConv2D)                                                       
                                                                 
 batch_normalization_8 (Batc  (None, 32, 32, 6)        24        
 hNormalization)                                                 
                                                                 
 leaky_re_lu_8 (LeakyReLU)   (None, 32, 32, 6)         0         
                                                                 
 max_pooling2d_6 (MaxPooling  (None, 16, 16, 6)        0         
 2D)                                                             
                                                                 
 separable_conv2d_7 (Separab  (None, 16, 16, 12)       138       
 leConv2D)                                            

In [19]:
# параметры данной ячейки могут быть изменены для получения более высокой точности
model.compile(
    optimizer=tf.keras.optimizers.Adam(
        learning_rate=tf.keras.optimizers.schedules.ExponentialDecay(initial_learning_rate=LEARNING_RATE, decay_steps=1000, decay_rate=0.5)
    ),
    loss=tf.keras.losses.CategoricalCrossentropy(),
    metrics=['accuracy']
)

In [20]:
# обучения модели
model.fit(
    x=X_train,
    y=y_train,
    validation_data=(X_val, y_val),
    batch_size=BATCH_SIZE,
    callbacks=[
        tf.keras.callbacks.ModelCheckpoint(filepath="{epoch:02d}-{val_accuracy:.2f}.hdf5", save_best_only=True),
        
    ],
    use_multiprocessing=True,
    workers=8,
    epochs=30
)

Epoch 1/30
391/391 [==============================] - 17s 41ms/step - loss: 3.9889 - accuracy: 0.1228 - val_loss: 4.4315 - val_accuracy: 0.1157
Epoch 2/30
391/391 [==============================] - 15s 38ms/step - loss: 3.3163 - accuracy: 0.1997 - val_loss: 3.7742 - val_accuracy: 0.1666
Epoch 3/30
391/391 [==============================] - 15s 39ms/step - loss: 3.1064 - accuracy: 0.2377 - val_loss: 3.1848 - val_accuracy: 0.2311
Epoch 4/30
391/391 [==============================] - 15s 38ms/step - loss: 2.9534 - accuracy: 0.2663 - val_loss: 3.1077 - val_accuracy: 0.2464
Epoch 5/30
391/391 [==============================] - 15s 38ms/step - loss: 2.8324 - accuracy: 0.2900 - val_loss: 3.0817 - val_accuracy: 0.2573
Epoch 6/30
391/391 [==============================] - 15s 38ms/step - loss: 2.7329 - accuracy: 0.3108 - val_loss: 2.8460 - val_accuracy: 0.2959
Epoch 7/30
391/391 [==============================] - 15s 38ms/step - loss: 2.6516 - accuracy: 0.3279 - val_loss: 2.8694 - val_accuracy: